# Running the Post-Annotation Pipeline

After the Figure-Eight job has completed, it is time to download the job. Figure Eight offers multiple types of jobs: 
    full (returns the Full report containing every judgment), 
    aggregated (returns the Aggregated report containing the aggregated response for each row), json (returns the JSON report containing the aggregated response, as well as the individual judgments), gold_report (returns the Test Question report), workset (returns the Contributor report), source (returns a CSV of the source data uploaded to the job).


### 1. Run download_processes.py

To download the job and get working data, run download_processes.py. 
Below is an example of the computer prompted questions and responses.

In [1]:
from post_annotation_scripts.fig_eight_download import download
from post_annotation_scripts.save_annotations import download_csv
from post_annotation_scripts.relabel_annotations import relabel
from post_annotation_scripts.reshape_annotations import reshape
from post_annotation_scripts.rename_annotated import rename_annotated
from post_annotation_scripts.movie_raw_generator import move
from post_annotation_scripts.celltk_processes import run_celltk
import sys
from post_annotation_scripts.cut_raw_segments import cut_raw
from post_annotation_scripts.make_training_data import training
import os
import shutil

/Users/danielkim/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
key = input('What is your Figure Eight api_key? ')
job_type = input('What type of report? ')
id = input('What is the job id to download? ')
relabelq = str(input('Do you want to uniquely annotate? (y/n) '))
montageq = str(input('Is this a montage? (y/n) ' ))

newdir = 'job_' + str(id) + '/'
if not os.path.exists('./' + newdir):
    os.makedirs('./' + newdir)
os.chdir('./' + newdir)

What is your Figure Eight api_key? B8rH7ALgZ9Q9NTksAxyh
What type of report? full
What is the job id to download? 1280335
Do you want to uniquely annotate? (y/n) y
Is this a montage? (y/n) y


Uniquely annotated signifies if you want the cell labels to be labeled sequentially. Note that not 
all annotaters use the labels in order, which can create problems while running Deep Cell. 

download_processes.py will create a new directory called job_ and the job id (for example, job_1280335) and all of the pipeline outputs will be placed into this job folder.


### 2. Download Job Report from Figure Eight

Next, download_processes.py will call download( ) from fig_eight_download.py to download the specified
job report from Figure Eight. If the function ran successfully, the zipped report should be in the job folder as output.zip.

In [3]:
import requests
import subprocess
import sys

def download(key, job_type, id):
    url = "https://api.figure-eight.com/v1/jobs/{job_id}.csv?type={type_job}&key={api_key}"
    url = url.replace('type_job', job_type)
    url = url.replace('api_key', key)
    url = url.replace('job_id', str(id))
    command = 'curl -s -o /dev/null -w "%{http_code}" "' + url + '"'
    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    if '302' not in str(out):
        print('error with downloading pt1')
        return

    url2 = "https://api.figure-eight.com/v1/jobs/{job_id}.csv?type={type_job}&key={api_key}"
    url2 = url2.replace('type_job', job_type)
    url2 = url2.replace('api_key', key)
    url2 = url2.replace('job_id', str(id))
    command = 'curl -o "output.zip" -L "'
    command = command + url2 + '"'
    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()


In [4]:
print('-------------------------------------------------------------------------')
print('Downloading the job report from Figure Eight...')
download(key, job_type, id)

-------------------------------------------------------------------------


### 3. Unzip the Report and Download/Save Images from CSV

download_processes.py will call download_csv( ) from save_annotations.py to unzip
the job report and download and save the images from the csv into job folder 
under the annotations folder. The unzipped csv will also be available in the 
job folder under the unzipped_csv directory

In [5]:
from subprocess import call
import skimage.io
import skimage.measure
import scipy.misc
import numpy as np
import pandas as pd
import warnings
import pathlib
import os
import urllib.request, urllib.parse, urllib.error
import pdb

In [6]:
def download_csv():
    output_path = './annotations/'
    if not os.path.exists('./annotations/'):
        os.makedirs('./annotations')
    if not os.path.exists('./unzipped_csv/'):
        os.makedirs('./unzipped_csv')
    # Unzip csv
    call(["unzip", "output.zip", '-d', './unzipped_csv'])
    dir_path = './unzipped_csv'
    dirs = os.listdir(dir_path)
    csv = ''
    for file in dirs:
        if '.csv' in file:
            csv = file
    if csv == '':
        print('No csv found in unzipped_csv/')
        return

    csv_file = os.path.join('./unzipped_csv/', file)
    df = pd.DataFrame.from_csv(csv_file)

    urls = df.loc[:,['annotation', 'broken_link', 'image_url']]
    split_start = None
    # iterate through rows of .csv file
    for index, row in df.iterrows():

        if row['broken_link'] is False:
            # Get image_name
            annotation_url = row['annotation'][8:-2]
            image_url = row['image_url']

            # generate image id
            image_url_split = image_url.split("/")
            image_id = image_url_split[-1]
            lst = image_id.split('.png')
            image_id = lst[0]
            
            # pad labels with zeros to ensure labels are read sequentially
            if split_start == None:
                print(image_id)
                split_start = int(input('Index of start? '))
                
            if int(image_id[split_start:]) <= 9 and len(image_id[split_start:]) == 1:
                image_id = image_id[:split_start] + '0' + image_id[split_start:]

            annotated_image_folder = output_path
            if not os.path.exists(annotated_image_folder):
                os.makedirs(annotated_image_folder)

            annotated_image_name = "annotation_" + image_id
            annotated_image_path = os.path.join(annotated_image_folder, annotated_image_name)

            # Download annotated image
            annotated_image = urllib.request.URLopener()
            annotated_image.retrieve(annotation_url, annotated_image_path)


In [7]:
print('-------------------------------------------------------------------------')
print('Downloading annotations from job report...')
download_csv()

/Users/danielkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


-------------------------------------------------------------------------
montage_0_0
Index of start? 10


/Users/danielkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  This is separate from the ipykernel package so we can avoid doing imports until


Index 10 indicates the position of the second 0. By doing so, the montages, movies, and annotations will be put in order. 

### 4. Uniquely Annotate the Annotations

If opted for sequentially annotating the annotations, download_processes.py will call relabel( ) from relabel_annotations.py. The newly relabelled annotations will be saved into the job folder under the relabelled_annotations directory. Note that relabelling may take some time. If not, skip to step 6. 

In [9]:
import numpy as np
from skimage.io import imsave, imread
import matplotlib.pyplot as plt
import os
from skimage.morphology import remove_small_holes, remove_small_objects



In [10]:
def relabel():

    montage_path = './annotations/'
    list_of_montages = os.listdir(montage_path)
    output_path = './relabelled_annotations/'
    if os.path.isdir(output_path) is False:
        os.makedirs(output_path)

    for montage_name in list_of_montages:
        # get the image segment
        save_ind = os.path.splitext(montage_name)[0][len("annotation_"):]
        montage_file = os.path.join(montage_path, montage_name)
        img_array = imread(montage_file)[:,:,0]
        img_clean = clean_montage(img_array)
        seq_label = relabel_montage(img_clean)
        seq_img = "seq_annotation" + save_ind + ".tif"
        image_path = os.path.join(output_path, seq_img)
        imsave(image_path, seq_label.astype(np.uint8))

In [13]:
def clean_montage(img):
    clean_img = remove_small_holes(img, connectivity=1,in_place=False)
    clean_img = remove_small_objects(clean_img, min_size=10, connectivity=1, in_place=False)
    fixed_img = np.zeros(img.shape, dtype=np.uint8)
    # using binary clean_img, assign cell labels to pixels of new image
    # iterate through the entire image
    for x in range(1, img.shape[0]):
        for y in range(1, img.shape[1]):
            # if clean_img is true, that pixel should have a label in fixed_img
            if clean_img[x, y].any():
                # get possible labels from original image (exclude background)
                ball = img[x-1:x+1, y-1:y+1].flatten()
                ball = np.delete(ball, np.where(ball == 0))
                # if no possible labels
                if len(ball) == 0: 
                    if x>1 and y>1:
                        # take a larger area for labels, if possible
                        ball = img[x-2:x+2, y-2:y+2].flatten()
                        ball = np.delete(ball,np.where(ball == 0))
                        if len(ball) != 0:
                            # if there are possible values, take the most common
                            pixel_val = np.argmax(np.bincount(ball))
                            fixed_img[x, y] = pixel_val
                        else:
                            # otherwise take the label of that pixel from the original img
                            # output location & frame to for user reference
                            fixed_img[x,y] = img[x,y]
                            print("x, y: ", (x, y), "   frame: ", (((x//216)-1)*10+y//256))
                    else:
                        # otherwise take the label of that pixel from the original img
                        # output location & frame to for user reference
                        fixed_img[x,y] = img[x,y]
                        print("x, y: ", (x, y), "   frame: ", (((x//216)-1)*10+y//256))
                else: # if there are possible values, take the most common
                    pixel_val = np.argmax(np.bincount(ball))
                    fixed_img[x, y] = pixel_val
    return fixed_img

In [14]:
def relabel_montage(y):
    # create new_y to save new labels to
    new_y = np.zeros(y.shape)
    unique_cells = np.unique(y) # get all unique values of y
    unique_cells = np.delete(unique_cells, np.where(unique_cells == 0)) # remove 0, as it is background
    relabel_ids = np.arange(1, len(unique_cells) + 1)
    # iterate through existing labels, and save those pixels as new id in new_y
    for cell_id, relabel_id in zip(unique_cells, relabel_ids):
        cell_loc = np.where(y == cell_id)
        new_y[cell_loc] = relabel_id
    return new_y

In [15]:
if relabelq == 'y':
        print('Uniquely annotating the annotations...')
        relabel()

Uniquely annotating the annotations...


/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/morphology/misc.py:194: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  "Did you mean to use a boolean array?", UserWarning)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_4_01.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_2_04.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/

x, y:  (1, 1801)    frame:  -3
x, y:  (1, 1802)    frame:  -3


/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_1_01.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_3_03.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_3_01.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_2_02.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


x, y:  (862, 1606)    frame:  26
x, y:  (862, 1607)    frame:  26
x, y:  (863, 1606)    frame:  26
x, y:  (863, 1607)    frame:  26
x, y:  (863, 1608)    frame:  26
x, y:  (863, 1861)    frame:  27
x, y:  (863, 1862)    frame:  27
x, y:  (863, 1863)    frame:  27
x, y:  (863, 1864)    frame:  27
x, y:  (863, 1865)    frame:  27
x, y:  (863, 1866)    frame:  27


/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_4_02.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_2_03.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_0_04.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontage_1_03.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./relabelled_annotations/seq_annotationmontag

### 5. Convert Montages to Movies

Next, download_processes.py will call reshape( ) from reshape_annotations.py 
if the images are montages. Reshape will create chronological movies in the job
folder under the movie directory for each montage. Reshape will ask for a few inputs.
The answers to these questions can be easily found by opening up a montage in 
ImageJ.

In [17]:
from subprocess import call
import skimage.io
import skimage.measure
import scipy.misc
import numpy as np

import warnings
import pathlib
import os
import urllib.request, urllib.parse, urllib.error
import pdb

In [19]:
def reshape():
    montage_path = './relabelled_annotations/'
    output_path = './movie/'
    list_of_montages = os.listdir(montage_path)
    x_image = int(input('How many images down? '))
    y_image = int(input('How many images across? '))
    x_dim = int(input('X dim of montage: '))
    y_dim = int(input('Y dim of montage: '))
    buffer_size = int(input('Size of buffer? '))
    x_sizes = int((x_dim - ((x_image - 1) * buffer_size)) / x_image)
    y_sizes = int((y_dim - ((y_image - 1) * buffer_size)) / y_image)

    for montage_name in list_of_montages:
        if os.path.isdir(output_path) is False:
            os.makedirs(output_path)
        montage_file = os.path.join(montage_path, montage_name)
        subfolder = montage_name[14:-4]
        output_folder = os.path.join(output_path, subfolder)

        reshape_montage(montage_file, output_folder, x_size=x_sizes, y_size=y_sizes, x_images=x_image, y_images=y_image, buffer=buffer_size)


In [20]:
def reshape_montage(montage_file, output_folder, x_size = 256, y_size = 256, x_images = 3, y_images = 10, buffer=0):
    debug = False
    # open composite image
    img = skimage.io.imread(montage_file)
    # create output directory
    if os.path.isdir(output_folder) is False:
        os.makedirs(output_folder)

    # chop up the montage
    x_end = x_size - 1
    y_end = y_size - 1
    images = np.ndarray( shape=(x_size, y_size, x_images*y_images), dtype=np.int16)
    image_number = 0
    while x_end < (x_size*x_images + (x_images - 1) * buffer):
        # moving along columns until we get to the end of the column
        while y_end < (y_size*y_images + (y_images - 1) * buffer):
            if debug:
                print("x_end: " + str(x_end))
                print("y_end: " + str(y_end))
            images[:,:,image_number] = img[
                    (x_end-(x_size-1)):(x_end+1),
                    (y_end-(y_size-1)):(y_end+1)]
            image_number += 1
            y_end += y_size + buffer
        # once we reach the end of a column, move to the beginning of the
        # next row, and continue
        y_end = y_size - 1
        x_end += x_size + buffer

    # renumber the images so that the numbers are 1 to N
    labels = np.unique(images)
    images_copy = np.zeros(images.shape, dtype = np.uint8)
    for counter, label in enumerate(labels):
        if label != 0:
            images_copy[np.where(images == label)] = counter
    images = images_copy
    # save images
    with warnings.catch_warnings():
        for image in range(images.shape[-1]):
            image_output_dir = os.path.join(output_folder, 'annotated')
            if os.path.isdir(image_output_dir) is False:
                os.makedirs(image_output_dir)
            skimage.io.imsave(os.path.join(image_output_dir, str(image).zfill(2) + '.tif'), images[:,:,image])




In [23]:
print('-------------------------------------------------------------------------')
if montageq == 'y':
    print('Reshaping the annotation images... ')
    reshape()

-------------------------------------------------------------------------
Reshaping the annotation images... 
How many images down? 4
How many images across? 10
X dim of montage: 864
Y dim of montage: 2560
Size of buffer? 0


/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_01/annotated/00.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_01/annotated/01.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_01/annotated/02.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_01/annotated/03.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_01/annotated/04.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_0_00/annotated/05.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_0_00/annotated/06.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_0_00/annotated/07.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_0_00/annotated/08.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_0_00/annotated/09.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_03/annotated/10.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_03/annotated/11.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_03/annotated/12.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_03/annotated/13.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_03/annotated/14.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_04/annotated/17.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_04/annotated/18.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_04/annotated/19.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_04/annotated/20.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_04/annotated/21.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

...

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_00/annotated/31.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_00/annotated/32.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_00/annotated/33.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_00/annotated/34.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_00/annotated/35.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_01/annotated/36.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_01/annotated/37.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_01/annotated/38.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_4_01/annotated/39.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_0_04/annotated/00.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_2_04/annotated/00.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_2_04/annotated/01.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_2_04/annotated/02.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_2_04/annotated/03.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_2_04/annotated/04.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_02/annotated/06.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_02/annotated/07.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_02/annotated/08.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_02/annotated/09.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_02/annotated/10.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_03/annotated/11.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_03/annotated/12.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_03/annotated/13.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_03/annotated/14.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danielkim/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: ./movie/montage_3_03/annotated/15.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/dan

### 6. Cut Raw Images and Move to Movie Folder

Next, cut_raw from cut_raw_segments.py will be called. This will cut up raw data images and turn them also into movies. After which, the new 
images will be put into the movies folder of the job folder. Some inputs will 
be asked of the function, including the directory of where the raw data folder is 
in relation to the deepcell-data-engineering directory, the channel names, and the number of sets. 
Note that most jobs will be made for one set at a time

Steps 6 and 7 wll not be able to be run from the Jupyter Notebook and must be 
done in the docker file to output properly.

In [80]:
from annotation_scripts.utils import get_image, get_images_from_directory
import numpy as np
import skimage as sk
import os
from scipy import ndimage
import scipy
import argparse
import pdb

In [86]:
def cut_raw():
    base_direc = str(input('Directory to data folder (e.g. /data/data/cells/HeLa/S3)'))
    channel_names = str(input('What channels are there? '))
    channel_names = channel_names.split(', ')

    set_number = int(input('Set number: '))
    part_num = int(input('Part number (if N/A, type -1): '))
    if part_num != -1:
        base_part = str(input('Part base name (e.g. montage_part_)'))
    num_segs = int(input('Number of segments to make in x/y direction (i.e. 4 --> 4x4): '))
    data_subdirec = "raw"
    save_stack_subdirec = "stacked_raw"

    # load images
    for channel_name in zip(channel_names):
        if part_num != -1:
            path_image = 'set' + str(set_number) + '/' + base_part + str(part_num)
        else:
            path_image = "set" + str(set_number)
        direc = os.path.join(base_direc, path_image)
        directory = os.path.join(direc, data_subdirec)

        images = get_images_from_directory(directory, [channel_name[0]])

        number_of_images = len(images)

        image_size = images[0].shape

        crop_size_x = image_size[1]//num_segs
        crop_size_y = image_size[2]//num_segs

        # make directory for stacks of cropped images if it does not exist
        stacked_direc = os.path.join(direc, save_stack_subdirec)
        if os.path.isdir(stacked_direc) is False:
            os.mkdir(stacked_direc)

        # crop images
        for i in range(num_segs):
            for j in range(num_segs):
                list_of_cropped_images = []
                for stack_number in range(number_of_images):
                    img = images[stack_number][0,:,:,0]
                    cropped_image = img[i*crop_size_x:(i+1)*crop_size_x, j*crop_size_y:(j+1)*crop_size_y]

                    raw_image_name = ''

                    if part_num != -1:
                        raw_image_name = 'set_' + str(set_number) + '_' + base_part + str(part_num)
                    else:
                        raw_image_name = 'set_' + str(set_number)
                    cropped_image_name = raw_image_name + '_x_' + str(i) + '_y_' + str(j) + '_slice_' + str(stack_number) + '.png'
                    cropped_folder_name = os.path.join(direc, save_stack_subdirec, raw_image_name + '_x_' + str(i) + '_y_' + str(j))
                    # make directory if it does not exit
                    if os.path.isdir(cropped_folder_name) is False:
                        os.mkdir(cropped_folder_name)
                    # save stack of a segment over time
                    cropped_image_name = os.path.join(cropped_folder_name, cropped_image_name)
                    scipy.misc.imsave(cropped_image_name, cropped_image)

In [82]:
import sys
import os
import shutil

In [83]:
def move(job_id):
    data_folder = str(input('Relative path from deepcell-data-engineering directory to stacked_raw folder in raw data folder: '))
    data_folder = './data/set0/stacked_raw/'
    dir_list = os.listdir(data_folder)
    destination = './job_' + str(job_id) + '/movie/montage_'
    for term in dir_list:
        i = term.split('x_')[1][0]
        j = term.split('y_')[1][0]
        files = os.listdir(data_folder + term)
        if not os.path.exists(destination + str(i) + '_0' + str(j) + '/raw/'):
            os.makedirs(destination + str(i) + '_0' + str(j) + '/raw/')
        for file in files:
            if file.endswith('.png'):
                shutil.copy(data_folder + term + '/' + file, destination + str(i) + '_0' + str(j) + '/raw/')


In [84]:
print('-------------------------------------------------------------------------')
print('Cutting raw images and moving them to movie folder...')
data_path = str(input('Path to data folder with raw images: '))
os.chdir('../' + data_path)

-------------------------------------------------------------------------
Cutting raw images and moving them to movie folder...
Path to data folder with raw images: 


In [ ]:
cut_raw()
move(id)
os.chdir('./' + newdir)
# Output Below:

Directory to data folder (e.g. /data/data/cells/HeLa/S3)./data/

What channels are there? Far-red

Set number: 0

Part number (if N/A, type -1): -1

Number of segments to make in x/y direction (i.e. 4 --> 4x4): 5
./data/set0/raw ('Far-red',)

Relative path from deepcell-data-engineering directory to stacked_raw folder in raw data folder: ./data/set0/stacked_raw/ 

### 7. Make Training Data

Finally, download_processes.py will call training() from make_training_data.py.
This will create npz files containing the training data into the job folder. Training
data can be made for Patchwise sampling, Fully convolutional training of single image conv-nets, and fully 
convolutional training of movie conv-nets. 

In [89]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
import os
import pathlib
import skimage as sk
import scipy as sp
from scipy import ndimage
from skimage import feature
from sklearn.utils import class_weight
from annotation_scripts.utils import get_image
from skimage import morphology as morph
import matplotlib.pyplot as plt
from skimage.transform import resize
from annotation_scripts.data_utils import make_training_data

def training():
    # Define maximum number of training examples
    window_size = 30

    # Load data
    direc_name = './movie'
    output_directory = './'
    training_data_name = str(input('Training data filename: '))
    file_name_save = os.path.join(output_directory, training_data_name + '.npz')
    training_direcs = os.listdir(direc_name)
    channel_names = ["set_0"]

    # Create output ditrectory, if necessary
    pathlib.Path( output_directory ).mkdir( parents=True, exist_ok=True )

    # Create the training data
    make_training_data(window_size_x = 30, window_size_y = 30,
        direc_name = direc_name,
        montage_mode=False,
        file_name_save = file_name_save,
        training_direcs = training_direcs,
        channel_names = channel_names,
        dimensionality = 3,
        annotation_name = "",
        raw_image_direc = "raw",
        annotation_direc = "annotated",
        border_mode = "same",
        output_mode = "conv",
        num_frames = 40,
        reshape_size = None,
        display = False,
        num_of_frames_to_display = 5,
        verbose = True)


In [ ]:
print('----------------------------------------------------------------------------')
print('Making deepcell training data...')
training()

Example output: 

Making deepcell training data...

Training data filename: training_data

Loading set_0 channel data from training dir 1: ./movie/montage_3_00/raw
Skipping final 6 frames, as num_frames is 40 but there are 46 total frames

Loading set_0 channel data from training dir 2: ./movie/montage_2_01/raw
Skipping final 6 frames, as num_frames is 40 but there are 46 total frames

Loading set_0 channel data from training dir 3: ./movie/montage_1_00/raw
Skipping final 6 frames, as num_frames is 40 but there are 46 total frames

...

Skipping final 6 frames, as num_frames is 40 but there are 46 total frames
Loading set_0 channel data from training dir 25: ./movie/montage_4_01/raw
